This programming exercise is from the textbook [Think Complexity, 2nd edition](https://thinkcomplex.com) by Allen Downey. This book is distributed under the [MIT License](http://opensource.org/licenses/MIT).

Some computer code from the textbook were also reused and modified for the purposes of this exercise. These reused computer code are indicated in the solution for this exercise and are still credited to the author.

**Exercise:** My implementation of `reachable_nodes_bfs` is efficient in the sense that it is in $O(n + m)$, but it incurs a lot of overhead adding nodes to the queue and removing them.  NetworkX provides a simple, fast implementation of BFS, available from [the NetworkX repository on GitHub](https://github.com/networkx/networkx/blob/master/networkx/algorithms/components/connected.py).

Here is a version I modified to return a set of nodes:

In [2]:
def plain_bfs(G, start):
    """A fast BFS node generator"""
    seen = set()
    nextlevel = {start}
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G[v])
    return seen

Compare this function to `reachable_nodes_bfs` and see which is faster.  Then see if you can modify this function to implement a faster version of `shortest_path_dijkstra`

# Solution goes here

Included here is the solution for exercise 3.1 which is used to test the efficiency of `reachable_nodes_bfs` and `plain_bfs` from the textbook. Note that these two functions are largely unmodified except for added spaces in order to increase readability.

In [1]:
import networkx as nx

def adjacent_edges(nodes, k):
    n = len(nodes)
    
    if k%2 != 0 and n%2 != 0:                                            # Checks if n is odd and k is odd
        raise ValueError('k cannot be odd when n is odd')                # Raises the value error
    
    else:
        if k % 2 == 0:                                                   # This part is similar to the one in the book
            k = k//2
            for i in range(n):
                for j in range(i+1, i+k+1):
                    yield nodes[i],nodes[j%n]                            # Except that it uses the actual values of the nodes
                
        else:                                                            # Runs this part if k is odd
            k = k-1                                                      # Turns odd k into even k and repeats the above code
            k = k // 2
            for i in range(n):
                for j in range(i+1-k, i+k+1):
                    yield nodes[i],nodes[j%n]
                
                yield nodes[i],nodes[(i+n//2)%n]                         # This adds an edge to the opposite node

def make_regular_graph(node, k):
    G = nx.Graph()                                                       # Creates the initial graph G
    
    G.add_nodes_from(range(node))
    G.add_edges_from(adjacent_edges(range(node),k))                             # Runs the adjacent_edges function
    
    return G

These next two functions `reachable_nodes_bfe` and `plain_bfs` are the ones provided by the textbook

In [2]:
from collections import deque

def reachable_nodes_bfs(G, start):
    seen = set()
    queue = deque([start])
    
    while queue:
        node = queue.popleft()
        
        if node not in seen:
            seen.add(node)
            queue.extend(G.neighbors(node))
            
    return seen

In [3]:
def plain_bfs(G, start):
    seen = set()
    nextlevel = {start}
    
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        
        for v in thislevel:
            if v not in seen:
                seen.add(v)
                nextlevel.update(G[v])
                
    return seen

In [4]:
complete = make_regular_graph(1000, 10)

In [5]:
%timeit len(reachable_nodes_bfs(complete, 0))

1.17 ms ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [6]:
%timeit len(plain_bfs(complete, 0))

1.13 ms ± 5.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


It can be seen here that `plain_bfs` performs faster than `reachable_nodes_bfs`.

In [7]:
def modified_djikstra(G, start):
    dist = {}
    nextlevel = {start}
    value = 0
    
    while nextlevel:
        thislevel = nextlevel
        nextlevel = set()
        
        for v in thislevel:
            if v not in dist:
                """To add new entries  to dictionaries use dict_name[key] = value"""
                
                dist[v] = value
                nextlevel.update(G[v])
                
        value += 1
                
    return dist

Here `modified_djistra` is the `plain_bfs` from the textbook modified to perform Djikstra's algorithm. In order to compare this to `shortest_path_djikstra`, the function from the textbook must be copied here:

In [8]:
def shortest_path_dijkstra(G, source):
    """Finds shortest paths from `source` to all other nodes.
    
    G: graph
    source: node to start at
    
    returns: make from node to path length
    """
    dist = {source: 0}
    queue = deque([source])
    while queue:
        node = queue.popleft()
        new_dist = dist[node] + 1

        neighbors = set(G[node]).difference(dist)
        for n in neighbors:
            dist[n] = new_dist
        
        queue.extend(neighbors)
    return dist

The following tests whether they provided the same shortest path.

In [9]:
complete_1 = make_regular_graph(6, 3)

d1 = modified_djikstra(complete_1, 0)
d2 = shortest_path_dijkstra(complete_1, 0)

d1==d2

True

This shows that they provide the same shortest paths. 

In [10]:
%timeit len(modified_djikstra(complete_1, 0))

6.56 µs ± 33.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
%timeit len(shortest_path_dijkstra(complete_1, 0))

7.57 µs ± 61.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


This shows that `modified_djikstra` performs faster than `shortest_path_djikstra`